# imports

In [1]:
import requests
from fake_useragent import UserAgent
import time
from bs4 import BeautifulSoup
from datetime import date
import requests
from requests import get
import pandas as pd
today = date.today()

# crawler

## defining url

In [2]:
def scrape_web(url):    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    ua=UserAgent()
    hdr = {'User-Agent': ua.random,
          'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
          'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
          'Accept-Encoding': 'none',
          'Accept-Language': 'en-US,en;q=0.8',
          'Connection': 'keep-alive'}
    time.sleep(2)
    response = requests.get(url, headers=hdr)
    time.sleep(4)
    html_soup = BeautifulSoup(response.content,"lxml")   
    return html_soup

In [3]:
url = 'https://www.pararius.com/apartments/nederland/page-'
html_soup = scrape_web(url+'1')

## get numbers to define iteractions

In [4]:
num_pages = html_soup.find_all('a',{"class":"pagination__link"})[4].text
num_pages = int(num_pages)

num_rents = html_soup.find_all('div',{"class":"pagination__summary"})[0].text
num_rents = int(num_rents.split()[-2])

In [5]:
print(num_pages)
print(num_rents)

174
5220


## get total html soup

In [6]:
houses = []
for count in range(1, num_pages+1):
    try:
        html_soup = scrape_web(url + str(count))        
        house_data = html_soup.find_all('li', class_="search-list__item search-list__item--listing")
        houses.append(house_data)
    except:
        print('erro')
        pass

## get data

In [7]:
price_=[]
address_=[]
street_=[]
agency_=[]
irl_=[]
image_=[]
description_2=[]

for n_pages in range (num_pages):
    n_pages_len = len(houses[n_pages])
    for n_rents in range (n_pages_len):
        num = (houses[n_pages][n_rents])
        list_li = num.find_all('li',{"class":"illustrated-features__item"})
        description_1={}
        
        try:
            p5_value = num.find_all('span',{"class":"listing-label listing-label--under-option"})[0].text
        except:
            p5_value = None
        
        for i in range(len(list_li)):
            a = (list_li[i]["class"][1][28:])
            b = (list_li[i].text)
            description_1.update({
                a:b,
                'status':p5_value,
            })
        description_2.append(description_1)
        
        price = num.find_all('div',{"class":"listing-search-item__price"})[0].text
        price_.append(price)

        address = num.find('div',{"class":"listing-search-item__location"}).text
        address_.append(address)

        street = num.find_all('a',{"class":"listing-search-item__link listing-search-item__link--title"})[0].text
        street_.append(street)

        irl = num.find_all('a',{"class":"listing-search-item__link listing-search-item__link--title"})[0]['href']
        irl_.append(irl)
        
        image = num.find_all('img')[0]['src']
        image_.append(image)

        agency = num.find_all('a', href=True)[2].text
        agency_.append(agency)

# Into DataFrame

In [8]:
df_pararius_1 = pd.DataFrame({
    'price':price_,
    'address':address_,
    'street':street_,
    'agency':agency_,
    'irl':irl_,
    'image':image_,
})
df_pararius_2 = pd.DataFrame(description_2)

## Unify dataframes

In [9]:
df_pararius = pd.concat([df_pararius_1, df_pararius_2],1)

/var/folders/pl/wm7wjyvn1vvflk70yn1b6n6c0000gn/T/ipykernel_15903/2613757866.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  df_pararius = pd.concat([df_pararius_1, df_pararius_2],1)


# fixing text

In [10]:
df_pararius['surface-area']        = df_pararius['surface-area'].str.replace("\D","",regex=True)
df_pararius['number-of-rooms']     = df_pararius['number-of-rooms'].str.replace("\D","",regex=True)
df_pararius['garden-surface-area'] = df_pararius['garden-surface-area'].str.replace("\D","",regex=True)
df_pararius['plot-size']           = df_pararius['plot-size'].str.replace("\D","",regex=True)

In [11]:
df_pararius = df_pararius.dropna(axis=1, how='all')
df_pararius['price'] = df_pararius['price'].str.replace("\D","",regex=True)
df_pararius['address'] = df_pararius['address'].str.replace('\n|new|  ',"",regex=True)
df_pararius['postcode'] = df_pararius['address'].str.replace("\s","",regex=True).str[0:6]
df_pararius['date'] = str(today)

In [12]:
df_pararius['status'] = df_pararius['status'].str.replace('\n',"",regex=True)

In [13]:
df_pararius = df_pararius.drop_duplicates(subset=['irl']).reset_index(drop=True)

# save data

In [14]:
df_pararius

,price,address,street,agency,irl,image,surface-area,status,number-of-rooms,interior,construction-period,garden-surface-area,plot-size,postcode,date
0,1264,"3572 PK Utrecht (Zeeheldenbuurt, Hengeveldstra...",Apartment Willem Barentszstraat,Simpelsearch,/apartment-for-rent/utrecht/70050cb0/willem-ba...,https://media.pararius.nl/image/PR0001658000/P...,42,None,2,Furnished,NaN,NaN,NaN,3572PK,2021-11-01
1,1195,3037 TD Rotterdam (Liskwartier),Apartment Willebrordusplein,Perfectrent,/apartment-for-rent/rotterdam/940dddf5/willebr...,https://casco-media-prod.global.ssl.fastly.net...,53,None,2,Upholstered,NaN,NaN,NaN,3037TD,2021-11-01
2,1045,3811 JW Amersfoort (Coninckstraat),Apartment Oliesteeg,HouseHunting Utrecht,/apartment-for-rent/amersfoort/2cbb4088/oliesteeg,https://casco-media-prod.global.ssl.fastly.net...,49,None,2,Upholstered,NaN,NaN,NaN,3811JW,2021-11-01
3,1500,3514 BH Utrecht (Vogelenbuurt),Apartment Gruttersdijk,Spot Makelaardij,/apartment-for-rent/utrecht/0ed3d6e9/gruttersdijk,https://casco-media-prod.global.ssl.fastly.net...,51,None,2,NaN,2010,NaN,NaN,3514BH,2021-11-01
4,1285,1971 DM IJmuiden (Lagersbuurt),House Gjertsenstraat,123Wonen Haarlem,/house-for-rent/ijmuiden/25c2dcaa/gjertsenstraat,https://casco-media-prod.global.ssl.fastly.net...,74,None,4,Upholstered,NaN,NaN,NaN,1971DM,2021-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4796,1500,1505 TJ Zaandam (Bomenbuurt),House Lijsterbesstraat,Uw Partner in Onroerende Zaken,/house-for-rent/zaandam/1b23bcc5/lijsterbesstraat,https://casco-media-prod.global.ssl.fastly.net...,86,Under option,4,Upholstered,NaN,NaN,NaN,1505TJ,2021-11-01
4797,3450,3045 LJ Rotterdam (Zestienhoven),House Overschiese Kleiweg,BenHousing,/house-for-rent/rotterdam/e5d0ae4e/overschiese...,https://casco-media-prod.global.ssl.fastly.net...,294,Under option,6,Upholstered or furnished,NaN,NaN,NaN,3045LJ,2021-11-01
4798,960,3762 GK Soest (Klaarwater),Apartment Wiardi Beckmanstraat,Regio Vastgoedmakelaars,/apartment-for-rent/soest/2e33b708/wiardi-beck...,https://casco-media-prod.global.ssl.fastly.net...,90,Under option,4,Shell,NaN,NaN,NaN,3762GK,2021-11-01
4799,1350,1567 LD Assendelft (Parkrijk),Apartment Wisselwachter,Uw Partner in Onroerende Zaken,/apartment-for-rent/assendelft/829433fa/wissel...,https://casco-media-prod.global.ssl.fastly.net...,76,Under option,3,Upholstered,NaN,NaN,NaN,1567LD,2021-11-01


In [20]:
today_csv = ("df_pararius_"+ str(today)+'.csv')
df_pararius.to_csv(f'data/temp/{today_csv}')